## Uvod u Znanost o podacima - Sara Sicic

------------------------------

### Članak: Metabolism-associated molecular classiﬁcation of hepatocellular carcinoma

Karcinom jetre jedan je od najkompliciranijih karcinoma zbog velike raznolikosti gena koji su ekspresirani u uzorcima karcinoma. To znatno otežava i analizu podataka i liječenje pacijenata. Cilj članka je podijeliti uzorke karcinoma jetre u tri grupe (klastera), od kojih svaka grupa ima određene karakteristike, npr. dobra prognoza ili loša prognoza, te na temelju 90 metaboličkih gena predvidjeti kojoj grupi karcinom pripada.

------------------------------


In [1]:
#importi
import sys
#!{sys.executable} -m pip install gtfparse
#!{sys.executable} -m pip install tabulate
#!{sys.executable} -m pip install xlrd
#!{sys.executable} -m pip install lifelines
#!{sys.executable} -m pip install scipy
#!{sys.executable} -m pip install combat --user
#!{sys.executable} -m pip install scikit-survival
#!{sys.executable} -m pip install xlrd
#!{sys.executable} -m pip install kneed

In [2]:
import pandas as pd
import numpy as np
from gtfparse import read_gtf
import csv
from tabulate import tabulate
import os
import gzip
import xlrd
import shutil
from scipy import stats
from os import listdir
from os.path import isfile, join
from combat.pycombat import pycombat
from sklearn.decomposition import PCA

### 1) Priprema podataka

Podatke sam sve stavila u folder "files", unzippala sam ih i ostavila originalna imena, jedino sam tcga podatke extractala u folder "tcga". Ako ćete pokretati sa svog računala zamjenite path u idućem dijelu sa svojim.

In [3]:
path = 'C:/Users/Sara/Documents/My Fax/UZOP/files/'

In [4]:
#metabolicki geni
df = pd.ExcelFile(path + '41586_2011_BFnature10350_MOESM321_ESM.xls').parse('All metabolic genes') #you could add index_col=0 if there's an index
gene_set=set()
for cell in df['Gene Symbol']:
    gene_set.add(cell)
    

In [5]:
#anotacije
annot = read_gtf(path + 'gencode.v22.annotation.gtf')
gene_dict_id = dict()
gene_dict_len = dict()

gene_set_2 = set()
for i in range (0, len(annot)):
    if annot["gene_name"][i].upper() in gene_set:
        if ((annot["end"][i]-annot["start"][i])!=0):
            gene_set_2.add(annot["gene_name"][i])
            gene_dict_id[annot["gene_id"][i]]=annot["gene_name"][i]
            gene_dict_len[annot["gene_name"][i]]=annot["end"][i]-annot["start"][i]
            

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_status', 'gene_name', 'level', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_status', 'transcript_name', 'tag', 'transcript_support_level', 'havana_transcript', 'exon_number', 'exon_id', 'ont', 'protein_id', 'ccdsid']


In [6]:
#icgc ucitavanje
file_icgc = open(path + 'exp_seq.tsv')
icgc_df = pd.read_csv(file_icgc, delimiter="\t")
icgc_df = icgc_df.loc[:, ['icgc_donor_id','gene_id','raw_read_count']]
print(icgc_df)

        icgc_donor_id         gene_id  raw_read_count
0             DO50855            MPP2            3608
1             DO50855  TMEM189-UBE2V1          446395
2             DO50855            ELF3         1275154
3             DO50855            SPEG            6104
4             DO50855         PTPN20A              59
...               ...             ...             ...
8342109       DO23508          PEX11G           47800
8342110       DO23508          CHTF18           35983
8342111       DO23508          TXNDC6            1958
8342112       DO23508          ZNF33A          166682
8342113       DO23508       LOC283481           10453

[8342114 rows x 3 columns]


In [7]:
icgc_df = icgc_df[icgc_df['gene_id'].isin(gene_set_2)]
print(icgc_df)

        icgc_donor_id  gene_id  raw_read_count
14            DO50855    SGPP1          193710
58            DO50855  SLC37A2           16555
70            DO50855  SLC6A11           27485
72            DO50855     DERA          130239
77            DO50855  PLA2G4E            1477
...               ...      ...             ...
8342083       DO23508    COX8C              34
8342091       DO23508    KCNC3            3516
8342094       DO23508   SLC3A1           16897
8342099       DO23508    ACAD8          146259
8342100       DO23508     PFKP           17030

[1070527 rows x 3 columns]


In [8]:
#icgc tpm

icgc_count = dict()
icgc_data = []

for index, row in icgc_df.iterrows():
    rrc0 = float(row['raw_read_count'])
    genelen = float(gene_dict_len[row['gene_id']])
    rrc = rrc0/genelen
    icgc_data.append([row['icgc_donor_id'], row['gene_id'], rrc])
    
    if row['icgc_donor_id'] not in icgc_count:
        icgc_count[row['icgc_donor_id']]=rrc
    else:
        icgc_count[row['icgc_donor_id']]+=rrc         
        
for row in icgc_data:
    row[2] = row[2] * 1000000 / icgc_count[row[0]]
    
        
icgc_df = pd.DataFrame(icgc_data, columns=['donor_id','gene_id','raw_read_count'])

print(icgc_df)

        donor_id  gene_id  raw_read_count
0        DO50855    SGPP1        3.874531
1        DO50855  SLC37A2        0.285830
2        DO50855  SLC6A11        2.827911
3        DO50855     DERA       45.659909
4        DO50855  PLA2G4E        0.112523
...          ...      ...             ...
1070522  DO23508    COX8C        0.003774
1070523  DO23508    KCNC3        0.159332
1070524  DO23508   SLC3A1        2.772726
1070525  DO23508    ACAD8        2.993089
1070526  DO23508     PFKP        3.124463

[1070527 rows x 3 columns]


In [9]:
#tcga ucitavanje

dir_tcga = os.listdir(path + 'tcga')
n=0
data = []
for dir in dir_tcga:
    n += 1
    dirName = path + 'tcga/'+dir
    for f in listdir(dirName):
        if (f.endswith('gz')):
            file = dirName+'/'+ f
            patientName = f[0:len(f)-16]
            tcgaFile = pd.read_csv(file, compression='gzip', sep='\t')
            for index, row in tcgaFile.iterrows():
                geneid = tcgaFile.iloc[index,0]
                if geneid in gene_dict_id:
                    if gene_dict_id[geneid] in gene_set_2:
                        data.append([patientName, gene_dict_id[geneid], tcgaFile.iloc[index,1] ])
    if (n==5):
        break
        
tcga_df = pd.DataFrame(data, columns=['donor_id','gene_id','raw_read_count'])
print(tcga_df)

                                   donor_id  gene_id  raw_read_count
0      3243c347-c219-487f-87fa-ec4046feee81     DPM1            1200
1      3243c347-c219-487f-87fa-ec4046feee81    FUCA2            3719
2      3243c347-c219-487f-87fa-ec4046feee81     GCLC            4159
3      3243c347-c219-487f-87fa-ec4046feee81    ENPP4             271
4      3243c347-c219-487f-87fa-ec4046feee81     CFTR               2
...                                     ...      ...             ...
13260  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d    DHRSX               0
13261  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d    ASMTL               0
13262  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d  SLC25A6               0
13263  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d   PLCXD1               0
13264  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d     ASMT               0

[13265 rows x 3 columns]


In [10]:
#tcga tpm

tcga_count = dict()
tcga_data = []

for index, row in tcga_df.iterrows():
    rrc0 = float(row['raw_read_count'])
    genelen = float(gene_dict_len[row['gene_id']])
    rrc = rrc0/genelen
    tcga_data.append([row['donor_id'], row['gene_id'], rrc])
    
    if row['donor_id'] not in tcga_count:
        tcga_count[row['donor_id']]=rrc
    else:
        tcga_count[row['donor_id']]+=rrc         

for row in tcga_data:
    row[2] = row[2] * 1000000 / tcga_count[row[0]]
    
        
tcga_df = pd.DataFrame(tcga_data, columns=['donor_id','gene_id','raw_read_count'])

print(tcga_df)

                                   donor_id  gene_id  raw_read_count
0      3243c347-c219-487f-87fa-ec4046feee81     DPM1      155.269356
1      3243c347-c219-487f-87fa-ec4046feee81    FUCA2       21.749406
2      3243c347-c219-487f-87fa-ec4046feee81     GCLC      538.137711
3      3243c347-c219-487f-87fa-ec4046feee81    ENPP4        0.091643
4      3243c347-c219-487f-87fa-ec4046feee81     CFTR        0.014477
...                                     ...      ...             ...
13260  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d    DHRSX        0.000000
13261  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d    ASMTL        0.000000
13262  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d  SLC25A6        0.000000
13263  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d   PLCXD1        0.000000
13264  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d     ASMT        0.000000

[13265 rows x 3 columns]


In [11]:
icgc_df_new = icgc_df.rename(columns={"icgc_donor_id": "donor_id"})
result_df = pd.concat([icgc_df_new, tcga_df], axis=0)
result_df.fillna(0)
#result_df['gene_id_and_donor_id'] = result_df['gene_id'] + result_df['donor_id']
print(result_df)

                                   donor_id  gene_id  raw_read_count
0                                   DO50855    SGPP1        3.874531
1                                   DO50855  SLC37A2        0.285830
2                                   DO50855  SLC6A11        2.827911
3                                   DO50855     DERA       45.659909
4                                   DO50855  PLA2G4E        0.112523
...                                     ...      ...             ...
13260  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d    DHRSX        0.000000
13261  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d    ASMTL        0.000000
13262  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d  SLC25A6        0.000000
13263  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d   PLCXD1        0.000000
13264  0ab21a73-a03d-4ef6-954b-460e7ca4ae6d     ASMT        0.000000

[1083792 rows x 3 columns]


In [12]:
pivoted_df = result_df.reset_index().pivot_table(index=["gene_id"], columns=["donor_id"], aggfunc='first')
dropped_df = pivoted_df.fillna(0)
print(dropped_df)

                                        index  \
donor_id 0ab21a73-a03d-4ef6-954b-460e7ca4ae6d   
gene_id                                         
A1CF                                  12044.0   
A4GALT                                11558.0   
A4GNT                                 11404.0   
AACS                                  10876.0   
AADAC                                 11319.0   
...                                       ...   
XYLB                                  10955.0   
XYLT1                                 11089.0   
XYLT2                                 10684.0   
ZACN                                  12943.0   
ZADH2                                 12794.0   

                                                                              \
donor_id 3243c347-c219-487f-87fa-ec4046feee81  DO227643  DO227801    DO23508   
gene_id                                                                        
A1CF                                   1432.0  634065.0  655774.0  106701

In [13]:
icgc_pivot = icgc_df_new.reset_index().pivot_table(index=["gene_id"], columns=["donor_id"], aggfunc='first')
tcga_pivot = tcga_df.reset_index().pivot_table(index=["gene_id"], columns=["donor_id"], aggfunc='first')

In [14]:
#pycombat
datasets = [icgc_pivot, tcga_pivot]

for j in range(len(datasets)):
    batch.extend([j for _ in range(len(datasets[j].columns))])

df_corrected = pycombat(dropped_df,batch)



NameError: name 'batch' is not defined

### 2) Odabir bitnih gena i klasteriranje

In [ ]:
#MAD
#>0.5
import scipy as sp
n = 0
counts_mad = dict()
for index, row in dropped_df.iterrows():
    for r in row:
        if index not in counts_mad:
            counts_mad[index] = [r]
        else:
            counts_mad[index].append(r)
for key in counts_mad:
    if (sp.stats.median_abs_deviation(counts_mad[key])>0.5):
        new_genes.add(key)
mad_df = dropped_df.loc[dropped_df.index.isin(new_genes)]
#mad_df = result_df.loc[result_df['gene_id'].isin(new_genes)]
#print(mad_df)
#mad_up_df =  mad_df.pivot(id_vars=['index'], var_name='donor_id', value_name='rrc') 
#mad_up_df = pd.melt(mad_df, value_vars=['donor_id', 'rrc'])
mad_t_df = mad_df.T
#mad_up_df = mad_df.pivot_table(index=["donor_id"], columns=["gene_id"], aggfunc='first')
print(mad_t_df)

In [ ]:
#uctavanje survival podataka
dfile1 = open(path + 'donor.tsv')
icgc_donor = pd.read_csv(dfile1, delimiter="\t")
dfile2 = open(path + 'clinical.tsv')
tcga_donor = pd.read_csv(dfile2, delimiter="\t")

    
survivalDict = dict()
icgc_donor_df = icgc_donor.loc[:,['icgc_donor_id', 'donor_vital_status', 'donor_survival_time']];
icgc_donor_df.rename(columns={'icgc_donor_id':'donor_id', 'donor_vital_status':'status', 'donor_survival_time':'time'}, inplace=True)
tcga_donor_df = tcga_donor.loc[:,['case_id', 'vital_status', 'days_to_death']];
tcga_donor_df.rename(columns={'case_id':'donor_id', 'vital_status':'status', 'days_to_death':'time'}, inplace=True)

donor_df = icgc_donor_df
donor_df = pd.concat([icgc_donor_df,tcga_donor_df], axis=0)
donor_df = donor_df.set_index('donor_id')
#print(mad_t_df.index)
mad_t_df = mad_t_df.reset_index().set_index('donor_id')

drop_mad_df = mad_t_df[mad_t_df.index.isin(donor_df.index)]
drop_donor_df = donor_df[donor_df.index.isin(mad_t_df.index)]
drop_mad_df = drop_mad_df[drop_mad_df.index.isin(drop_donor_df.index)]
#donor_df = donor_df.drop_duplicates()
print("donor", drop_donor_df.shape,"mad T", mad_t_df.shape,"cetr", drop_mad_df.shape)
print(drop_mad_df.index)
print('----')
print(drop_donor_df.index)
drop_mad_df = drop_mad_df[~drop_mad_df.index.duplicated(keep='first')]
donor_df = pd.concat([drop_mad_df, drop_donor_df], axis=1, join='inner')
print("==========")
print(donor_df)

Nisam uspjela pronaci koji stupac u tcga_donor bi odgovarao vremenu prezivljavanja pacijenta do smrti.set_index

In [ ]:
from lifelines import CoxPHFitter

coxphf = CoxPHFitter(penalizer=0.9)
cpf_donor_df = donor_df.reset_index().drop(['donor_id', 'level_0', 'index'], axis=1)
cpf_donor_df["status"] = cpf_donor_df["status"].map(lambda x: 0 if x == 'alive' else 1)
coxphf.fit(cpf_donor_df, event_col='status', duration_col='time')


In [ ]:
c=0
for param in coxphf.params_:
    if (param>0):
        c+=1
print(c)

In [ ]:
#ucitavanje objavljenih gena

geneFile =  pd.ExcelFile('C:/Users/Sara/Documents/My Fax/UZOP/files/objavljeni_geni.xls')
sheet = geneFile.parse('Table S1') 
genes = dict()
for cell in sheet.iloc[1:, 0]:
    genes[cell] = 0


In [ ]:
#KMeans importi

import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans


In [ ]:
#KMeans
km_df = mad_df.T
print(km_df.shape)
model = KMeans(
    init="k-means++",
    n_clusters=3,
    n_init=4,
    max_iter=300,
    )

#kmeans.fit(km_df)
#id_clust = kmeans.fit_predict(km_df)
#print(id_clust)

In [ ]:
pca = PCA(n_components = 2)
data = pca.fit_transform(km_df)
label = model.fit_predict(km_df)
print(label)
plt.figure(figsize=(10,10))
uniq = np.unique(label)
for i in uniq:
   plt.scatter(data[label == i , 0] , data[label == i , 1] , label = i)

### 3)	Klasifikacija pomoću 90 gena
Zadnji korak u ovom projektu je treniranje klasifikatora koji na temelju 90 najznačajnijih gena određuje kojem klasteru (C1, C2, ili C3) pripada karcinom

In [ ]:
#feature selection algoritam
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

select_df = SelectKBest(chi2, k=90).fit_transform(mad_df.T, label)
print(select_df)


In [ ]:
#klasifikator SVM

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
y = np.array([1, 1, 2, 2])
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(select_df, label)
print(clf.predict(select_df))

In [ ]:
#prikaz rezultata
